In [1]:
import sys
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score, make_scorer
from imblearn.over_sampling import RandomOverSampler
from sklearn import model_selection

#Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

sys.path.insert(0, '/Users/gracewang/Documents/GitHub/elecfinal')
# sys.path.insert(0, 'D:\Fall23 Coursework\ELEC478\Competition\elecfinal')

In [2]:
from ml_pipeline import train_n_predict, validation, clean_split

In [3]:
from Data.data_cleaner import cleaner

In [5]:
## Clean data


train_path = "../Data/train_data.csv"
feature_path = "../Data/feature_weights.csv"
morph_path = "../Data/morph_embeddings.csv"
X_train, X_val, X_query, y_train, y_val, y_query = clean_split(train_path, feature_path, morph_path)

            ID  adp_dist  post_skeletal_distance_to_soma  \
0        42593   304.185                        353043.0   
1        42594   725.431                        244156.0   
2        42595  3423.030                        363829.0   
3        42596  3442.390                        344267.0   
4        42597  4442.380                        313630.0   
...        ...       ...                             ...   
185827  228420  3162.000                         86928.2   
185828  228421  3503.660                        114862.0   
185829  228422  3939.260                        140923.0   
185830  228423  4192.360                        110553.0   
185831  228424  4838.850                        363943.0   

        pre_skeletal_distance_to_soma  pre_oracle  pre_test_score  \
0                           1182170.0    0.831680        0.704509   
1                            914243.0    0.831680        0.704509   
2                           1171820.0    0.831680        0.704509   
3  

In [9]:
valid_X = X_val.select_dtypes(include='number').drop("ID", axis = 1)
valid_y = y_val

Using Validation

In [10]:
"""
    Function that outputs a model with optimal hyperparameters
    based on a validation set using grid search

    Inputs:
    model: provided model
    param_grid: dictionary of parameters and values to validate on
    e.g. 
    {'C': [0.001,0.01,0.1,1,10], 
    'gamma':[0.1,1,10,100], 
    'kernel':('linear', 'rbf')}
    valid_X: validation X of data (pandas df)
    valid_y: validation y of data

    Outputs: 
    clf: provided model with optimum hyperparameters
    """
pre_valid_models = [RandomForestClassifier(), LinearDiscriminantAnalysis()]
param_grids = [
    {'max_features': ['sqrt', 'log2'],
     'criterion' :['gini', 'entropy']},
     {'shrinkage': [None, 0.5],
      'solver':['svd','eigen']}
]
post_valid_models = []
for i in range(len(pre_valid_models)):
    best_clf = validation(model = pre_valid_models[i], 
                                        param_grid = param_grids[i], 
                                        valid_X = valid_X, 
                                        valid_y = valid_y)
    post_valid_models.append(best_clf)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
5 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/L

Training and Predicting

In [11]:
query_X = X_query.select_dtypes(include='number').drop("ID", axis = 1)
query_y = y_query

train_X = X_train.select_dtypes(include='number').drop("ID", axis = 1)
train_y = y_train

In [9]:
train_X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120329 entries, 0 to 185831
Data columns (total 14 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   adp_dist                        120329 non-null  float64
 1   post_skeletal_distance_to_soma  120329 non-null  float64
 2   pre_skeletal_distance_to_soma   120329 non-null  float64
 3   pre_oracle                      120329 non-null  float64
 4   pre_test_score                  120329 non-null  float64
 5   post_oracle                     120329 non-null  float64
 6   post_test_score                 120329 non-null  float64
 7   pre_nucleus_id                  120329 non-null  int64  
 8   post_nucleus_id                 120329 non-null  int64  
 9   fw_similarity                   120329 non-null  float64
 10  nuclei_adp_dist                 120329 non-null  float64
 11  ADP_total                       120329 non-null  float64
 12  connect_total        

In [10]:
"""
    Function that takes in a dataframe of data and outputs 
    a fitted "optimal" model

    Inputs:
    - train: training set
    - query: query set
    - models: dictionary of (model_name : model function) to train and predict on, with optimized 
    parameters already.

    Outputs:
    - best_clf: The optimum classifier function fitted over training data

    - accuracy_score: list of accuracies based on order of models
    passed.
    """

models = {"random forest": post_valid_models[0], "lda": post_valid_models[1]}
accuracy_score, best_clf = train_n_predict(train_X, train_y, query_X, query_y, models)

In [ ]:
leaderboard_path = "../Data/leaderboard_data.csv"
sub_data = cleaner(leaderboard_path, feature_path, morph_path, submission = True)
sub_data

,ID,adp_dist,post_skeletal_distance_to_soma,pre_skeletal_distance_to_soma,pre_oracle,pre_test_score,post_oracle,post_test_score,compartment,pre_brain_area,...,pre_feature_weights,post_feature_weights,post_morph_embeddings,fw_similarity,projection_group,axonal_coords,dendritic_coords,pre_nucleus_coords,post_nucleus_coords,nuclei_adp_dist
0,0,1.569857,1.066950,-1.520389,0.371616,-0.365468,0.880813,0.454349,apical_shaft,AL,...,"[0.19683373, 0.19128098, 0.49339047, -0.029781...","[-0.03441501, 0.41964892, 0.23311342, 0.327173...","[1.132591724395752, -0.0064769592136144, -0.67...",-1.613455,AL->V1,"[527499, 544761, 912282]","[529457, 543459, 916958]","[1368640, 451392, 861240]","[566528, 777344, 920800]",3.221214
1,1,1.072992,1.548930,0.949414,0.371616,-0.365468,1.170133,-0.223748,axon,AL,...,"[0.19683373, 0.19128098, 0.49339047, -0.029781...","[-0.03917461, -0.1830603, -0.3704222, 0.183293...","[1.0723994970321655, -0.7540942430496216, 0.11...",1.856578,AL->RL,"[1269890, 471870, 837816]","[1271230, 469651, 841045]","[1368640, 451392, 861240]","[1196352, 492736, 919560]",-0.818251
2,2,0.283729,-0.977701,-1.010610,0.371616,-0.365468,-0.889101,0.590201,basal,AL,...,"[0.19683373, 0.19128098, 0.49339047, -0.029781...","[-0.66653997, 0.5036552, 0.047438737, 0.181578...","[-0.0201839506626129, -0.2563340067863464, 0.1...",-0.923427,AL->V1,"[666057, 469875, 925239]","[665387, 467214, 923430]","[1368640, 451392, 861240]","[669760, 470336, 913720]",2.449958
3,3,0.531271,-0.742668,-0.920656,0.371616,-0.365468,-0.889101,0.590201,apical,AL,...,"[0.19683373, 0.19128098, 0.49339047, -0.029781...","[-0.66653997, 0.5036552, 0.047438737, 0.181578...","[-0.0201839506626129, -0.2563340067863464, 0.1...",-0.923427,AL->V1,"[696633, 453516, 925806]","[696648, 453417, 921875]","[1368640, 451392, 861240]","[669760, 470336, 913720]",2.283731
4,4,1.334973,-0.020021,-1.344421,0.371616,-0.365468,0.581479,0.526492,basal,AL,...,"[0.19683373, 0.19128098, 0.49339047, -0.029781...","[-0.40308818, 0.20905656, 0.25083357, -0.19731...","[-0.0254557952284812, 0.7573301792144775, -0.2...",-1.314208,AL->V1,"[567525, 585921, 888426]","[570714, 583676, 891264]","[1368640, 451392, 861240]","[672384, 609728, 932720]",3.030948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42588,42588,-0.620528,-0.368073,-0.917464,0.430932,0.065340,-0.040713,-0.247062,basal,V1,...,"[0.4097931, 0.2948376, 0.05925691, 0.111899965...","[-0.2947989, -0.35604966, 0.7178404, 0.1323682...","[1.4674314260482788, 0.6272068619728088, 0.410...",-0.329380,V1->V1,"[765513, 818622, 870114]","[766346, 817181, 871011]","[741376, 572288, 837120]","[807168, 757376, 869080]",-0.024980
42589,42589,0.073549,0.489645,-0.608341,0.430932,0.065340,-0.040713,-0.247062,apical,V1,...,"[0.4097931, 0.2948376, 0.05925691, 0.111899965...","[-0.2947989, -0.35604966, 0.7178404, 0.1323682...","[1.4674314260482788, 0.6272068619728088, 0.410...",-0.329380,V1->V1,"[784476, 609000, 828177]","[782564, 606911, 828562]","[741376, 572288, 837120]","[807168, 757376, 869080]",-1.069079
42590,42590,0.101911,-0.629141,-1.047202,0.430932,0.065340,-0.040713,-0.247062,basal,V1,...,"[0.4097931, 0.2948376, 0.05925691, 0.111899965...","[-0.2947989, -0.35604966, 0.7178404, 0.1323682...","[1.4674314260482788, 0.6272068619728088, 0.410...",-0.329380,V1->V1,"[766038, 773808, 861693]","[763428, 773490, 862976]","[741376, 572288, 837120]","[807168, 757376, 869080]",-0.270275
42591,42591,0.923462,0.536486,-0.695932,0.430932,0.065340,-0.040713,-0.247062,apical,V1,...,"[0.4097931, 0.2948376, 0.05925691, 0.111899965...","[-0.2947989, -0.35604966, 0.7178404, 0.1323682...","[1.4674314260482788, 0.6272068619728088, 0.410...",-0.329380,V1->V1,"[738276, 614460, 855099]","[741617, 612387, 854028]","[741376, 572288, 837120]","[807168, 757376, 869080]",-1.130779


In [ ]:
lb_data = sub_data.select_dtypes(include='number')

In [ ]:
connect_info = data[["ID","ADP_total", "connect_total","connect_rate"]]
lb_data = lb_data.merge(connect_info, how = 'left', left_on="ID", right_on="ID")
lb_data.info()

In [ ]:
#create a boolean prediction solution
lb_data["connected"] = best_clf.predict(lb_data.drop("ID", axis = 1))

In [ ]:
submission_data = lb_data.filter(['ID','connected'])

In [ ]:
submission_data.to_csv('submission_data.csv',index=False)